Materi ini diadaptasi dari

- Simon Rogers dan Mark Girolami. First Course in Machine Learning (2nd Ed). Bab 1

# Regresi linear

Kita akan mulai pembahasan kita dengan meninjau kembali permasalahan
regresi linear.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

Bagian ini opsional (untuk membuat agar format default dari Matplotlib adalah SVG)

In [ ]:
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

Bagian ini juga opsional:

In [ ]:
#matplotlib.style.use("ggplot")
matplotlib.style.use("dark_background")
matplotlib.rcParams.update({
    "axes.grid" : True,
    "grid.color": "gray",
    "grid.linestyle": "--"
})

Misalkan kita telah mengetahui bahwa data kita berasal dari fungsi
berikut:
$$
f(x) = 2x + 1
$$
Fungsi ini akan didefinisikan dengan menggunakan kode Python berikut:

In [ ]:
def f_true(x):
    return 2*x + 1

In [ ]:
f_true(1.1)

Pada kenyataannya, fungsi ini akan dipengaruhi oleh noise. Kita akan mensimulasikan noise
ini dengan bilangan acak yang terdistribusi normal dengan rata-rata 0 dan deviasi
standard $\sigma$ (atau amplitudo `A_noise`).
Anda dapat mencoba mengubah-ubah nilai `A_noise` ini.
Secara matematis dapat dituliskan sebagai
$$
\begin{align}
y = f(x) + \epsilon \\
\epsilon \sim \mathcal{N}(0,\sigma)
\end{align}
$$


Selain itu, kita juga memilih sampel titik-titik $x$ berada pada interval $[-5,5]$ dengan
jarak pisah yang seragam. Kita dapat menggunakan fungsi `np.linspace` untuk keperluan ini.

In [ ]:
xx = np.random.randn(5000)
plt.hist(xx, bins=50);

In [ ]:
np.random.seed(1234) # nilai awal random generator, useful for reproducibility
Npoints = 10
x = np.linspace(-5.0, 5.0, Npoints)
A_noise = 2.4 # standard deviation or amplitude of noise
y = f_true(x) + A_noise*np.random.randn(Npoints)
plt.scatter(x, y);
plt.grid(True)

Berikut adalah "data" yang kita miliki:

In [ ]:
import pandas as pd
pd.DataFrame( list(zip(x, y)) )

Misalkan sekarang kita ingin mencocokkan fungsi linear $\hat{f}(x)$ untuk data ini:
$$
\hat{f}(x) = w_0 + w_1 x
$$ {#eq-model-linear-01}
dengan $w_0$ dan $w_1$ adalah parameter model.

Sekarang permasalahan yang dihadapi adalah bagaimana memilih parameter
model yang terbaik. Sebelum itu, kita perlu mendefinisikan suatu kuantitas
atau metrik atau ukuran yang dapat dijadikan indikator seberapa baik model
untuk dataset yang diberikan.
Salah satu metrik yang dapat digunakan adalah beda kuadrat antara prediksi
model dan data target. Untuk data ke-$n$ dapat dituliskan sebagai berikut.
$$
\mathcal{L}_n \equiv \left( y_n - f(x_n; w_0, w_1) \right)^2
$$

Dengan merata-ratakan kontribusi dari seluruh data, dapat diperoleh
$$
\mathcal{L} = \frac{1}{N} \sum_{n=1}^{N} \mathcal{L}_n =
\frac{1}{N} \sum_{n=1}^{N} \left( y_n - f(x_n; w_0, w_1) \right)^2
$$ {#eq-loss-function-01}
Kuantitas ini dikenal sebagai fungsi rugi atau *loss function*.

Perhatikan bahwa fungsi rugi selalu bernilai positif atau sama dengan nol.
Jika kuantitas ini bernilai nol maka model yang digunakan dapat memberikan
prediksi yang tepat sama dengan data target. Secara praktis hal ini jarang
ditemui. Meskipun demikian, kita ingin fungsi rugi bernilai sekecil mungkin
sehingga prediksi model tidak jauh berbeda dengan target.
Untuk menemukan nilai fungsi rugi yang sekecil mungkin kita dapat melakukan
proses yang dikenal sebagai minimisasi atau optimasi fungsi.

Secara umu, kita dapat mencari parameter $(w_{0},w_{1})$ dengan menggunakan
prosedur minimisasi
yang secara matematis dapat dituliskan sebagai berikut.
$$
\arg\min_{w_{0},w_{1}} \frac{1}{N} \sum_{n=1}^{N} \mathcal{L}_{n}
$$

Untuk kasus khusus fungsi rugi pada Persamaan (@eq-loss-function-01), kita dapat
menemukannya secara analitik dengan menggunakan prinsip kalkulus: cari turunan pertama
dari  $\mathcal{L}$ terhadap $w_0$ dan $w_1$, kemudian mencari solusi dari:
$$
\begin{align*}
\frac{\partial\mathcal{L}}{\partial w_0} & = 0 \\
\frac{\partial\mathcal{L}}{\partial w_1} & = 0
\end{align*}
$$
Untuk fungsi rugi yang lebih umum, biasanya ekspresi yang dihasilkan tidak mudah
untuk diselesaikan secara analitik. Pada kasus ini biasanya digunakan
metode numerik seperti metode penurunan gradien (*gradient descent*).

Kita mulai dengan menuliskan fungsi rugi (@eq-loss-function-01) sebagai
$$
\begin{align*}
\mathcal{L} & = \frac{1}{N} \sum_{n=1}^{N} \left( t_n - (w_0 + w_1 x_{n}) \right)^2 \\
& = \frac{1}{N} \sum_{n=1}^{N} \left( w_1^2 x_n^2 + 2w_{1}x_{n}(w_0 - t_n) +
w_0^2 - 2w_0 t_n + t_n^2 \right)
\end{align*}
$$

Kemudian cari turunan dari $\mathcal{L}$ terhadap
$w_0$, $w_1$ dan menyamakan mereka dengan nol:
$$
\begin{align*}
\frac{\partial\mathcal{L}}{\partial w_1} & = 2w_1 \frac{1}{N} \left( \sum_{n=1}^{N} x_n^2 \right) +
\frac{2}{N} \left( \sum_{n=1}^{N} x_{n} (w_0 - t_n) \right) = 0 \\
\frac{\partial \mathcal{L}}{\partial w_0} & = 2w_0 + 2w_1 \frac{1}{N} \left( \sum_{n=1}^{N} x_n \right) -
\frac{2}{N} \left( \sum_{n=1}^{N} t_n \right) = 0
\end{align*}
$$

Dengan menyelesaikan persamaan ini diperoleh:
$$
\begin{align}
\begin{split}
w_{1} & = \frac{\overline{xy} - \overline{x}\overline{y}}{\overline{x^2} - \overline{x}^2} \\
w_{0} & = \overline{y} - w_{1} \overline{x}
\end{split}
\end{align}
$$ {#eq-w0-w1-analitik}
di mana notasi *overline* digunakan untuk menyatakan nilai rata-rata:
$$
\begin{align*}
\overline{x} & = \frac{1}{N} \sum_{n=1}^{N} x_{n} \\
\overline{y} & = \frac{1}{N} \sum_{n=1}^{N} y_{n}
\end{align*}
$$

Kode Python berikut ini mengimplementasikan perhitungan pada Persamaan (@eq-w0-w1-analitik).
Selain itu, kode ini juga menghitung hasil prediksi yang diperoleh dengan model linear
pada Persamaan (@eq-model-linear-01) dan membuat plot perbandingan dengan data.

In [ ]:
ybar = np.mean(y)
xbar = np.mean(x)
xybar = np.mean(x*y)
x2bar = np.mean(x**2)
  
w1 = (xybar - xbar*ybar)/(x2bar - xbar**2)
w0 = ybar - w1*xbar
  
print("Model parameters:")
print("w0 = %18.10e" % w0)
print("w1 = %18.10e" % w1)
  
y_pred = w0 + w1*x # garis lurus
  
plt.clf()
plt.plot(x, y, marker="o", linewidth=0, label="data")
plt.plot(x, y_pred, marker="x", label="linear-fit")
plt.grid(True)
plt.legend()
plt.xlabel("x")
plt.ylabel("y");

Bandingkan hasil dari $w_0$ dan $w_1$ yang sudah diperoleh dengan model yang sebenarnya.
Apakah mereka sama persis, cukup berdekatan, atau sangat jauh berbeda?

# Notasi vektor dan matriks

Pada bagian ini, kita akan menuliskan kembali permasalahan regresi linear
pada bagian sebelum dengan menggunakan notasi matriks dan vektor. Notasi ini
akan memungkinkan kita untuk menuliskan solusi $w_0$ dan $w_1$ menjadi lebih
kompak dan dapat diperumum untuk masalah regresi linear yang lebih luas.

Kita mulai dengan mendefinisikan vektor input dan vektor parameter atau
bobot sebagai berikut.
$$
\mathbf{x}_{n} = \begin{bmatrix}
1 \\
x_{n}
\end{bmatrix}
,\,\,\,%
\mathbf{w} = \begin{bmatrix}
w_{0} \\
w_{1}
\end{bmatrix}
$$

Dengan menggunakan definisi ini, model linear pada (@eq-model-linear-01)
menjadi
$$
f(x_n; w_0, w_1) = \mathbf{w}^{\mathsf{T}} \mathbf{x}_{n}
\label{}
$$ {#eq-model-linear-02}

Ekspresi untuk fungsi rugi, (@eq-loss-function-01),
menjadi:
$$
\mathcal{L} = \frac{1}{N} \sum_{n=1}^{N} \left( t_{n} - \mathbf{w}^{\mathsf{T}}
\mathbf{x}_{n} \right)^2
$$

Vektor input dapat disusun menjadi suatu matriks:
$$
\mathbf{X} = \begin{bmatrix}
\mathbf{x}^{\mathsf{T}}_{1} \\
\mathbf{x}^{\mathsf{T}}_{2} \\
\vdots \\
\mathbf{x}^{\mathsf{T}}_{N}
\end{bmatrix} =
\begin{bmatrix}
1 & x_{1} \\
1 & x_{2} \\
\vdots & \vdots \\
1 & x_{N} \\
\end{bmatrix}
$$

Kita juga dapat mendefinisikan vektor target sebagai
$$
\mathbf{y} = \begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_N
\end{bmatrix}
$$

Dengan definisi ini, kita dapat menuliskan fungsi rugi menjadi:
$$
\mathcal{L} = \frac{1}{N} \left( \mathbf{y} - \mathbf{Xw} \right)^{\mathsf{T}}
\left( \mathbf{y} - \mathbf{Xw} \right)
$$
Dengan mengalikan suku-suku dalam tanda kurung diperoleh:
$$
\begin{align}
\mathcal{L} & = \frac{1}{N} \left(
\mathbf{y}^{\mathsf{T}} \mathbf{y} +
\left(\mathbf{Xw}\right)^{\mathsf{T}} \mathbf{Xw} -
\mathbf{y}\mathbf{Xw} -
\left(\mathbf{Xw}\right)^{\mathsf{T}} \mathbf{y}
\right) \\
& = \frac{1}{N} \left(
\mathbf{w}^{\mathsf{T}} \mathbf{X}^{\mathsf{T}} \mathbf{X} \mathbf{w} -
2 \mathbf{w}^{\mathsf{T}} \mathbf{X}^{\mathsf{T}}\mathbf{y} +
\mathbf{y}^{\mathsf{T}} \mathbf{y}
\right)
\end{align}
$$

Turunan dari $\mathcal{L}$ terhadap $\mathbf{w}$ adalah
$$
\frac{\partial \mathcal{L}}{\partial \mathbf{w}} = \frac{2}{N}
\mathbf{X}^{\mathsf{T}} \mathbf{X} \mathbf{w} -
\frac{2}{N} \mathbf{X}^{\mathsf{T}} \mathbf{y} = \mathbf{0}
$$

Untuk menentukan nilai terbaik dari $\mathbf{w}$ kita dapat menggunakan
prosedur yang sama, yaitu dengan mencari solusi dari
$$
\dfrac{\partial \mathcal{L}}{\partial \mathbf{w}} = \mathbf{0}
$$
sehingga diperoleh
$$
\mathbf{X}^{\mathsf{T}} \mathbf{Xw} = \mathbf{X}^{\mathsf{T}}\mathbf{y} 
$$
Dengan menyelesaikan persamaan ini untuk $\mathbf{w}$ akhirnya diperoleh
$$
\mathbf{w} = \left(\mathbf{X}^{\mathsf{T}}\mathbf{X} \right)^{-1} \mathbf{X}^{\mathsf{T}} \mathbf{y}
$$ {#eq-w-vektor}

Kode Python berikut akan mengimplementasikan Persamaan (@eq-w-vektor) untuk regresi linear.

`np.hstack` juga dapat digunakan untuk membangun matrix $\mathbf{X}$ 

In [ ]:
Ndata = len(x)
# Build the input matrix or design matrix
X = np.zeros((Ndata,2))
X[:,0] = 1.0
X[:,1] = x

In [ ]:
X

In [ ]:
# Calculate the model parameters
XtX = X.T @ X   # @ adalah operator perkalian matriks pada Numpy
XtXinv = np.linalg.inv(XtX) # hitung inverse:
w = XtXinv @ X.T @ y

print("Model parameters:")
print("w0 = %18.10e" % w[0])
print("w1 = %18.10e" % w[1])
  
y_pred = X @ w

Konfirmasi bahwa solusi $w_0$ dan $w_1$ yang diperoleh sama dengan solusi dari
Persamaan (@eq-w0-w1-analitik).

In [ ]:
plt.clf()
plt.plot(x, y, marker="o", linewidth=0, label="data")
plt.plot(x, y_pred, marker="x", label="linear-fit")
plt.grid(True)
plt.legend()
plt.xlabel("x")
plt.ylabel("y");

# Regresi linear dengan `np.linalg.lstsq`

Alternatif, menggunakan fungsi `np.linalg.lstsq`:

In [ ]:
X

In [ ]:
res = np.linalg.lstsq(X, y, rcond=None)

In [ ]:
res

In [ ]:
w, *_ = np.linalg.lstsq(X, y, rcond=-1)

In [ ]:
w

Coba bandingkan hasil yang diperoleh dari `np.linalg.lstsq`
dengan hasil dari Persamaan (@eq-w0-w1-analitik). 

# Regresi linear pada Scikit Learn

Kelas `LinearRegression`:

```python
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)

# Evaluate model performance
R2_score = model.score(x, y)
print("R2_score on training data: ", R2_score)

# Prediction on training data
y_pred = model.predict(x)
RMSE = np.sqrt(np.mean((y_pred - y)**2))
print("RMSE training: ", RMSE)
```

In [ ]:
X

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y);

In [ ]:
model.intercept_   # w0

In [ ]:
model.coef_  # w1

Kita juga bisa menggunakan vektor `x` langsung ketika memanggil `model.fit()`. Tetapi `x` perlu disesuaikan dimensinya atay *shape*-nya.

In [ ]:
x.shape

In [ ]:
xnew = x.reshape(-1,1)

In [ ]:
xnew.shape

In [ ]:
x

In [ ]:
x[:,np.newaxis]

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xnew, y);

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x[:,np.newaxis], y);

In [ ]:
model.coef_ # w1

In [ ]:
model.intercept_ # w0

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x.reshape(-1,1), y);

In [ ]:
x.reshape(-1,1)

Untuk membuat prediksi dari data baru atau data latih, kita dapat menggunakan metode `predict`:

In [ ]:
model.predict([[-5]])

In [ ]:
y[0]

In [ ]:
model.predict(x[:,np.newaxis])

In [ ]:
y

Hitung mean square error:

In [ ]:
np.mean( (model.predict(x[:,np.newaxis]) - y)**2 )

# Ekstensi untuk model linear lain

Bagaimana jika kita ingin menggunakan model yang lain? Misalnya:
$$
y = w_0 + w_1 x^{(1)} + w_2 x^{(2)}
$$
atau
$$
y = w_0 + w_1 x + w_2 x^2 
$$

Dapat ditunjukkan bahwa Persamaan (@eq-w-vektor) juga dapat digunakan untuk model-model
tersebut. Yang diperlukan ada ekstensi dari definisi $\mathbf{X}$ dan $\mathbf{w}$

Misalnya pada kasus model:
$$
y = w_0 + w_1 x^{(1)} + w_2 x^{(2)}
$$
dengan $x^{(1)}$ dan $x^{(2)}$ adalah dua variabel input atau fitur. Definisi dari matriks
$\mathbf{X}$ adalah:
$$
\mathbf{X} =
\begin{bmatrix}
1 & x^{(1)}_{1} & x^{(2)}_{1} \\
1 & x^{(1)}_{2} & x^{(2)}_{2} \\
\vdots & \vdots & \vdots \\
1 & x^{(1)}_{N} & x^{(2)}_{N}
\end{bmatrix}
$$
dan vektor $\mathbf{w}$ sekarang terdiri dari tiga elemen:
$$
\mathbf{w} = \begin{bmatrix}
w_{0} \\
w_{1} \\
w_{2}
\end{bmatrix}
$$
Pada kasus ini kita memiliki dua variabel fitur yaitu $x^{(1)}$ dan $x^{(2)}$.

Begitu pula untuk kasus model
$$
y = w_0 + w_1 x + w_2 x^2 
$$ {#eq-quadratic-polynom}
Permasalah pencocokan data pada persamaan ini sering disebut sebagai regresi kuadrat
karena menggunakan persamaan kuadrat. Meskipun demikian, pada literatur pembelajaran
mesin, model ini masing tergolong model linear, karena parameternya $w_0$, $w_1$,
dan $w_2$ memiliki pangkat satu.

Untuk model pada @eq-quadratic-polynom kita dapat mendefinisikan matriks input
sebagai:
$$
\mathbf{X} =
\begin{bmatrix}
1 & x_{1} & x^{2}_{1} \\
1 & x_{2} & x^{2}_{2} \\
\vdots & \vdots & \vdots \\
1 & x_{N} & x^{2}_{N}
\end{bmatrix}
$$
Perhatikan bahwa kolom ketiga $x^{2}_{i}$ dapat dianggap sebagai "fitur baru"
atau "fitur tambahan"
yang dibuat dari "input" atau "fitur awal/asli".
Secara umum, kita dapat membuat fitur-fitur baru dari fitur yang sudah ada
yang bisa berupa kombinasi linear ataupun fungsi atau transformasi dari fitur asli.
Dalam beberapa kasus, bisa saja model dengan fitur baru dapat memiliki performa
yang lebih baik daripada fitur asli.
Hal ini merupakan salah satu bentuk dari rekayasa fitur (*feature engineering*).

## Tugas (1)

Buat data sintetik yang dihasilkan `f_true` dengan model yang lebih
kompleks seperti polinomial (plus random noise).

- Gunakan Persamaan (@eq-w-vektor) atau `np.linalg.lstsq` atau
`LinearRegression` pada `sklearn`. Bandingkan hasil yang Anda peroleh dengan
nilai parameter asli pada `f_true`.

### Menggunakan lstsq

In [ ]:
def f_true(x):
    return 3*x**2 + 2*x + 1

np.random.seed(1234) # nilai awal random generator, useful for reproducibility
Npoints = 10
x = np.linspace(-5.0, 5.0, Npoints)
A_noise = 5.0 # standard deviation or amplitude of noise
y = f_true(x) + A_noise*np.random.randn(Npoints)
plt.scatter(x, y);
plt.grid(True)

In [ ]:
x

In [ ]:
Ndata = len(x)
X = np.zeros((Ndata,3))
X[:,0] = 1.0
X[:,1] = x
X[:,2] = x**2

In [ ]:
X

In [ ]:
w, *_ = np.linalg.lstsq(X, y, rcond=-1)
w

In [ ]:
NptsDense = 100 # jumlah lebih banyak daripada jumlah titik data
xdense = np.linspace(-5,5,NptsDense)
Xdense = np.zeros((NptsDense,3))
Xdense[:,0] = 1
Xdense[:,1] = xdense
Xdense[:,2] = xdense**2

In [ ]:
ydense = Xdense @ w # perkalian matriks

In [ ]:
plt.plot(xdense, ydense, color="blue", label="model")
plt.scatter(x, y, label="data")
plt.legend();

### Menggunakan sklearn

In [ ]:
def f_true(x):
    return 3*x**2 + 2*x + 1

np.random.seed(1234) # nilai awal random generator, useful for reproducibility
Npoints = 10
x = np.linspace(-5.0, 5.0, Npoints)
A_noise = 5.0 # standard deviation or amplitude of noise
y = f_true(x) + A_noise*np.random.randn(Npoints)
plt.scatter(x, y);
plt.grid(True)

In [ ]:
model = LinearRegression()

Catatan: bisa juga menggunakan `PolynomialFeatures`. Di sini kita akan lakukan secara manual.

In [ ]:
X = np.zeros((Ndata,2))
X[:,0] = x
X[:,1] = x**2

In [ ]:
X

In [ ]:
model.fit(X, y);

In [ ]:
model.coef_ # w1 dan w3

In [ ]:
model.intercept_ # w0

In [ ]:
NptsDense = 100
xdense = np.linspace(-5,5,NptsDense)
Xdense = np.zeros((NptsDense,2))
Xdense[:,0] = xdense
Xdense[:,1] = xdense**2

In [ ]:
ydense = model.predict(Xdense)

In [ ]:
plt.plot(xdense, ydense, color="magenta", label="model")
plt.scatter(x, y, label="data");

In [ ]:
MSE_loss = np.mean( (y - model.predict(X))**2 )
print("MSE loss = ", MSE_loss)

In [ ]:
model.score(X, y)

Untuk `sklearn`, kita juga bisa menggunakan `PolynomialFeatures` untuk mendapatkan fitur polinomial.

`StandardScaler` juga dapat digunakan untuk melakukan standardisasi fitur.

## Tugas (2)

Buat data sintetik dengan menggunakan polinomial derajat tiga ditambah
dengan random noise. Gunakan data $x$ pada rentang $[-5,5]$, misalnya.
Gunakan misalnya 20 titik data.

- Lakukan regresi dengan menggunakan model dengen polinomial berderajat
  1 sampai 9 dan hitung rugi atau loss pada setiap model dengan menggunakan
  (@eq-loss-function-01). Buat plot data vs hasil prediksi dari polinomial
  yang digunakan.
  
- Buat plot dengan sumbu-$x$ adalah derajat polinomial
  dan sumbu-$y$ adalah loss yang dihitung. Catatan:
  Loss dihitung dengan menggunakan parameter yang paling optimal, yaitu parameter
  yang diperoleh dari (@eq-w-vektor).
  Pada polinomial derajat berapakah nilai loss yang diperoleh paling kecil?
  Apakah polinomial derajat tiga memiliki nilai loss yang paling rendah?

# Data latih dan data uji

Pada pembelajaran mesin kita biasanya lebih tertarik pada prediksi model untuk
data di luar data training. Oleh karena itu, biasanya data yang kita miliki dibagi
sedikitnya menjadi dua kelompok:

- data latih (*training dataset*): data ini digunakan untuk mendapatkan parameter
  model, misalnya parameter $\mathbf{w}$ pada model linear.

- data uji (*test dataset*): data ini digunakan untuk mengevaluasi performa model

Pada kasus jaringan saraf tiruan, sering juga digunakan satu set data lain yaitu
data validasi (*validation dataset*) yang digunakan pada waktu pelatihan jaringan saraf tiruan
(ANN) untuk mengevaluasi
progress model pada saat pelatihan.

## Tugas (3)

Tinjau kembali Tugas (2). Sekarang untuk dari 20 titik yang ada,
pilih titik-titik yang berada pada $-1 \leq x \leq 1$ sebagai data uji
dan titik yang lain adalah data latih.

Anda dapat menggunakan kode berikut.
```python
import numpy as np
Npoints = 20
x = np.linspace(-5.0, 5.0, Npoints)
idx1 = x <= 1
idx2 = x >= -1
idx_test = idx1 & idx2 # combine the condition
idx_train = ~idx_test # Other data is training data

x_train = x[idx_train]
x_test = x[idx_test]

# do similarly for y
# [....]
```

Ulangi yang Anda lakukan pada Tugas (2):

- $\mathbf{w}$ sekarang dicari dengan hanya menggunakan data latih atau
  training data.

- Loss sekarang dihitung dengan menggunakan data uji

- Plot loss terhadap orde polinomial yang digunakan. Polinomial orde berapakah yang
  menghasilkan loss paling kecil?

## Penggunaan sklearn

Pada soal sebelumnya kita telah memilih daerah $-1 \leq x \leq 1$ sebagai data uji.
Untuk kasus yang lebih umum, biasanya data uji ini dipilih secara acak.
Pada `sklearn` kita dapat menggunakan fungsi `train_test_split`. Contoh:

```python
import numpy as np
from sklearn.model_selection import train_test_split
X, y = np.arange(10).reshape((5, 2)), range(5)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
```

## Pemilihan model (*model selection*)

Permasalahan pemilihan orde polinomial merupakan salah satu contoh
pemilihan model. Pemilihan model dapat dilakukan berdasarkan beberapa
kriteria. Pada kasus regresi, salah satu kriteria yang dapat digunakan
adalah nilai dari fungsi _loss_ pada data uji.
Metode lain yang juga sering digunakan adalah metode validasi silang
(_cross validation_).